In [29]:
import tensorflow as tf

In [30]:
import numpy as np
import pandas as pd

In [31]:
df = pd.read_csv('cal_housing_clean.csv')

In [32]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [33]:
X = df.drop('medianHouseValue',axis=1)

In [34]:
y  = df['medianHouseValue']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [37]:
from sklearn.preprocessing import MinMaxScaler

In [38]:
scaler = MinMaxScaler()

In [39]:
X_train_scaled  = scaler.fit_transform(X_train)

In [40]:
X_test_scaled = scaler.transform(X_test)

In [43]:
X_train.index

Int64Index([19085, 10494,  4706, 12330, 14792, 16144,  5200,  9261,  4076,
            14078,
            ...
             3208, 10440, 13105,  6081,  6449,  1271, 10287, 18745, 17485,
             6976],
           dtype='int64', length=14448)

In [44]:
X_train = pd.DataFrame(data = X_train_scaled,
                       columns = X_train.columns,
                       index = X_train.index)

In [47]:
X_test = pd.DataFrame(data = X_test_scaled,
                       columns = X_test.columns,
                       index = X_test.index)

In [48]:
df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [79]:
age = tf.feature_column.numeric_column('housingMedianAge') 
rooms = tf.feature_column.numeric_column('totalRooms') 
bedrooms = tf.feature_column.numeric_column('totalBedrooms') 
population = tf.feature_column.numeric_column('population') 
households = tf.feature_column.numeric_column('households') 
income = tf.feature_column.numeric_column('medianIncome') 

In [80]:
feat_cols = [age,rooms,bedrooms,population,households,income]

In [81]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,
                                                             y=y_train,
                                                             batch_size=10,
                                                             num_epochs=1000,
                                                             shuffle=True)

In [82]:
model = tf.estimator.DNNRegressor(hidden_units = [6,6,6],feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\PRAMEE~1\\AppData\\Local\\Temp\\tmp1ieic2qt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [83]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\PRAMEE~1\AppData\Local\Temp\tmp1ieic2qt\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow

In [84]:
evaluate_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,
                                                                    y = y_test,
                                                                    batch_size=10,
                                                                    num_epochs=1,
                                                                    shuffle=False)

In [85]:
pred = model.evaluate(evaluate_input_func)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-03T12:09:00Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\PRAMEE~1\AppData\Local\Temp\tmp1ieic2qt\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.87574s
INFO:tensorflow:Finished evaluation at 2021-06-03-12:09:01
INFO:tensorflow:Saving dict for global step 100

In [86]:
print(pred)

{'average_loss': 57510916000.0, 'label/mean': 209422.66, 'loss': 57657344000.0, 'prediction/mean': 0.7799569, 'global_step': 1000}


In [87]:
predict_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [88]:
pred_gen = model.predict(predict_input_func)


In [89]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\PRAMEE~1\AppData\Local\Temp\tmp1ieic2qt\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [90]:
predictions[0]['predictions']

array([0.70233643], dtype=float32)

In [91]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [92]:
from sklearn.metrics import mean_squared_error

In [93]:
mean_squared_error(y_test,final_preds)**0.5


239814.2961144501